# `Занятие 3.2: Основы Web-разработки`

#### `Сириус, смена "Алгоритмы и анализ данных" 2024`

#### `Алексеев Илья, ММП ВМК МГУ`

[Источник GitHub](https://github.com/mmp-practicum-team/mmp_practicum_fall_2024/tree/main/Seminars/11-intro-web)




### `Мотивация. Игрушечный пример`

Представим ситуацию: вы решили сложную задачу (алгоритм поиска, обучение нейросети и т.д.). В нашем случае [задача с литкода](https://leetcode.com/problems/longest-palindromic-substring/description/).


Содержимое файла `solution.py`:
```python
def find_palindrome(s: str) -> str:
    input_str = s
    is_pal = [
        [False] * len(input_str) for _ in input_str
    ]  # is_pal[i][j] == True iff input_str[i:j+1] is palindromic

    for i in range(len(input_str)):
        is_pal[i][i] = True

    ans = input_str[0]
    for substring_len in range(2, len(input_str) + 1):
        cur_res = None
        for i in range(len(input_str) - substring_len + 1):
            j = i + substring_len - 1
            tmp_res = (input_str[i] == input_str[j]) and (
                substring_len <= 3 or is_pal[i + 1][j - 1]
            )
            is_pal[i][j] = tmp_res
            if tmp_res and cur_res is None:
                cur_res = input_str[i : j + 1]
        if cur_res is not None:
            ans = cur_res
    return ans

```

Что если вы хотите поделиться с миром своей работой? Нужно создать интерфейс взаимодействия! Говоря точно, API (application programming interface). 

### `Python package`

Первое что необходимо сделать, это обернуть свое решение в Python API. Сделаем пакет, из которого можно импортировать функцию `find_palindrome`.


Организуем проект:
```bash
toy_example/
├── longest_palindrome
│   ├── __init__.py
│   └── solution.py
└── requirements.txt
```

Здесь:
- `longest_palindrome/solution.py` --- код с решением задачи
- `longest_palindrome/__init__.py` --- init-файл, который объявляет директорию `longest_palindrome` пакетом
- `requirements.py` --- зависимости, используемые внутри `solution.py`

Содержимое файла `__init__.py`:
```python
from .solution import find_palindrome

__all__ = ["find_palindrome"]
```

Пример использования:
```python
from longest_palindrome import find_palindrome

find_palindrome("ababad")
```

### `Command-line interface (CLI) / Terminal user interface (TUI)`

Другой способ взаимодействия с вашим решением --- через терминал. Как насчет того, чтобы реализовать следующий интерфейс:

```bash
voorhs@maibenben:~$ longest_palindrome --input-word "ababad"
> ababa
```

В точности такой интерфейс пока реализовать не сможем... Но вот, что можем сделать. Добавим в корень проекта файл `cli.py` со следующим содержимым:
```python
import argparse
from longest_palindrome import find_palindrome

def main():
    parser = argparse.ArgumentParser(description="Find the longest palindrome in a string.")
    parser.add_argument("--input-word", type=str, help="The input string")
    args = parser.parse_args()

    result = find_palindrome(args.input_word)
    print(f"Longest palindrome: {result}")

if __name__ == "__main__":
    main()
```

Структура проекта:
```bash
toy_example/
├── longest_palindrome
│   ├── __init__.py
│   └── solution.py
├── requirements.txt
└── cli.py
```

Теперь можем взаимодействовать с нашим приложением через терминал:
```bash
python cli.py --input-word "ababad"
```

Такой способ взаимодействия доступен более широкому кругу пользователей, нежели взаимодействие через python api. Для него не нужно знать python и уметь импортировать. Достаточно просто написать команду в терминале, который стоит на любом компьютере.

### `HTTP server`

Ещё более общим способом взаимодействия с приложением являются HTTP-запросы. Они работают в формате "request--response". Вот простой пример, реализующий такой интерфейс:
```python
from fastapi import FastAPI
from longest_palindrome import find_palindrome

app = FastAPI()

@app.get("/longest-palindrome/{input_string}")
def longest_palindrome(input_string: str):
    return find_palindrome(input_string)
    

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)
```

Теперь можно провзаимодействовать с приложением, введя в браузере следующий адрес
```url
http://127.0.0.1:8000/longest-palindrome/ababad
```
В ответ получим веб-страницу, с единственным содержимым: `"ababa"`

О других способах создания HTTP-запросов поговорим далее в этом уроке (python requests, curl). Единственный вывод, который нужно сделать из данного игрушечного примера: HTTP --- это интефейс взаимодействия с приложением (API). 

Разговору о FastAPI будет посвящена вся лекция, которая состоится через неделю.

### `Создание HTTP-запросов`

Пример с обращением через браузер был скорее для наглядности. В реальной жизни такой способ взаимодействия используют исключительно для получения медиа-контента в виде веб-страниц ("сайтов").

В более общем случае HTTP может передавать произвольную информацию. Чтобы её получать, можете воспользоваться командой `curl` в терминале или библиотекой `requests` в python.

Пример **CURL** (client URL).
```bash
curl 'http://127.0.0.1:8000/longest-palindrome/babad'
```

Пример `requests`:

```python
import requests

response = requests.get('http://127.0.0.1:8000/longest-palindrome/babad')

print(response.text)
```

### `HTTP`

HTTP (HyperText Transfer Protocol) — это протокол, позволяющий получать различные ресурсы. Изначально, как следует из названия — для документов, но сейчас уже для передачи произвольных данных. Лежит в основе обмена данными в Web.

Это протокол клиент-серверного взаимодействия, что означает инициирование запросов к сереверу самим получателем. Итоговый документ может состоять из различных частей: текст, аудио/видео файлов, скриптов. Взаимодействие осущетвляется посредством обмена одиночными сообщенимями: запрос-ответ.

**Утилиты для работы с HTTP есть в любом популярном языке программирования.** На нем основана web разработка, которая состоит из backend и frontend части. Backend предоставляет HTTP API, а frontend обращается к нему и визуализирует.


> Python: The requests library is widely used for making HTTP requests. Python also has built-in libraries like urllib and http.client for this purpose.
> 
> JavaScript (Node.js): Libraries like axios, fetch (in browser environments), and the built-in http and https modules are commonly used.
> 
> Java: Libraries such as HttpURLConnection, OkHttp, and Apache HttpClient are popular choices.
> 
> C# (.NET): The HttpClient class is commonly used for making HTTP requests.
> 
> Ruby: The Net::HTTP library is built-in, and Faraday is a popular third-party library.
> 
> PHP: The cURL extension and the Guzzle library are widely used.
> 
> Go: The net/http package is part of the standard library and is used for making HTTP requests.
> 
> Swift: Libraries like URLSession and third-party libraries like Alamofire are commonly used.
> 
> Kotlin/Java (Android): Libraries like Retrofit and OkHttp are popular for making HTTP requests.
> 
> Rust: The reqwest crate is commonly used for making HTTP requests.

HTTP является протоколом прикладного уровня, который в качестве транспорта использует возможности другого протокола — TCP. Основное требование к транспортному протоколу — надёжность, то есть, гарантированная доставка сообщений. По этой причине не подходит другой распространённый протокол — UDP, который не гарантирует доставку сообщений.

#### Преимущества
- Прост и человекочитаем
- Расширяем
- Не имеет состояния (каждый запрос — в отрыве от остальных), следовательно, базово простой

#### Расширения
- Кэш — сервер может инструктировать клиента/прокси о том, что и как надолго можно кэшировать
- Ослабления ограничения источника — инструкции клиенту о том, что на загружаемой странице может содержаться информация с других доменов
- Аутентификация — для доступа к защищённой информации
- Прокси и туннелирование — сокрытие источника или получателя информации, кэширование для уменьшения нагрузки
- Сессии — расширение для сохранения состояния взаимодействия. Несколько механизмов, самый известный — cookies

### `Пример с GitHub API`

Получить информацию о контрибьюторах репозитория:

```bash
curl -X GET \
    -H "Authorization: token $github_api_token" \
    https://api.github.com/repos/deeppavlov/AutoIntent/contributors
```

Получить информацию о ветках репозитория:
```bash
curl -X GET \
    -H "Authorization: token $github_api_token" \
    https://api.github.com/repos/deeppavlov/AutoIntent/branches
```

Открыть новый issue:
```bash
curl -X POST \
    -H "Authorization: token $github_api_token" \
    -d '{"title":"Found a bug","body":"Im having a problem with this."}' \
    https://api.github.com/repos/octocat/Hello-World/issues
```

Здесь `-X` называют типом (методом) запроса, `-d` называют телом запроса, `-H` --- заголовком запроса.

При работе с API почти всегда возникает необходимость авторизовываться с помощью токена (ключа). Личный токен не стоит распространять, если не хочется, чтобы кто-то выполнил действия от вашего имени.

В linux такие секретные ключи можно собрать в `.env` файл и загрузить как переменные окружения из терминала командой `. .env`.

Всё то же самое, но с помощью python `requests`:

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
github_api_token = os.environ["github_api_token"]

В питоне секретные ключи можно загрузить в linux-окружение с помощью third-party библиотекеи `dotenv`. Чтобы загрузить переменные из linux-окружения, достаточно обратиться к словарику `os.environ` стандартной библиотеки `os`.

In [3]:
headers = {
    "Authorization": f"token {github_api_token}"
}

In [4]:
import requests


# GET request for contributors
contributors_url = "https://api.github.com/repos/deeppavlov/AutoIntent/contributors"
contributors_response = requests.get(contributors_url, headers=headers)
print(contributors_response.json())

# GET request for branches
branches_url = "https://api.github.com/repos/deeppavlov/AutoIntent/branches"
branches_response = requests.get(branches_url, headers=headers)
print(branches_response.json())

# POST request to create an issue
issues_url = "https://api.github.com/repos/octocat/Hello-World/issues"
issue_data = {
    "title": "Found a bug",
    "body": "I'm having a problem with this."
}
issues_response = requests.post(issues_url, headers=headers, json=issue_data)
print(issues_response.json())

[{'login': 'Samoed', 'id': 36135455, 'node_id': 'MDQ6VXNlcjM2MTM1NDU1', 'avatar_url': 'https://avatars.githubusercontent.com/u/36135455?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/Samoed', 'html_url': 'https://github.com/Samoed', 'followers_url': 'https://api.github.com/users/Samoed/followers', 'following_url': 'https://api.github.com/users/Samoed/following{/other_user}', 'gists_url': 'https://api.github.com/users/Samoed/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/Samoed/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/Samoed/subscriptions', 'organizations_url': 'https://api.github.com/users/Samoed/orgs', 'repos_url': 'https://api.github.com/users/Samoed/repos', 'events_url': 'https://api.github.com/users/Samoed/events{/privacy}', 'received_events_url': 'https://api.github.com/users/Samoed/received_events', 'type': 'User', 'user_view_type': 'public', 'site_admin': False, 'contributions': 57}, {'login': 'voorhs', 'id': 4450

### `Пример с VK API`

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
app_service_token = os.environ["app_service_token"]

In [6]:
import requests

def get_newsfeed(query: str, app_service_token: str):
    return requests.get(
        url="https://api.vk.ru/method/newsfeed.search",
        params={
            "v": "5.199",
            "access_token": app_service_token,
            "q": query
        },
    ).json()

In [7]:
response = get_newsfeed("pepe", app_service_token)

In [16]:
item = response["response"]["items"][1]
list(item.keys())

['inner_type',
 'donut',
 'comments',
 'marked_as_ads',
 'type',
 'attachments',
 'date',
 'from_id',
 'id',
 'likes',
 'reaction_set_id',
 'owner_id',
 'post_source',
 'post_type',
 'reposts',
 'text',
 'views']

In [ ]:
from pprint import pprint

pprint(item["text"])

('Дата рождения\n'
 '12 декабря 1915 год\n'
 'Киноактёр, Кинорежиссёр, Певец \n'
 'Лауреат «Оскар»\n'
 'исполнил роли в более чем в 50 фильмах\n'
 '#Frank #Sinatra\n'
 'Ну, а теперь фильм 1947 год:\n'
 'Это произошло в Бруклине\n'
 'It Happened in #Brooklyn\n'
 '#Фрэнк #Синатра и #Джимми #Дюранте \n'
 'поют романс #ОчиЧёрные\n'
 '\n'
 'Потрясающая Фильмография\n'
 '1941 ф Ночи Лас-Вегаса Las Vegas Nights в роли самого себя\n'
 '1942 ф На судне Ship Ahoy в роли самого себя\n'
 '1943 ф Побудка с Беверли Reveille with Beverly в роли самого себя\n'
 '1943 ф Выше и выше Higher and Higher Фрэнк\n'
 '1944 кор — The Shining Future в роли самого себя\n'
 '1944 кор — The Road to Victory в роли самого себя\n'
 '1944 ф Шагай веселее Step Lively Гленн Расселл\n'
 '1945 ф Поднять якоря Anchors Aweigh Кларенс Дулиттл\n'
 '1946 ф Когда рассеиваются тучи Till the Clouds Roll By в роли самого себя\n'
 '1947 ф Это произошло в Бруклине It Happened in Brooklyn Дэнни Уэбсон Миллер\n'
 '1948 ф Чудо колокола 

In [93]:
content["orig_photo"]

{'height': 2560,
 'type': 'base',
 'url': 'https://sun1-13.vkuserphoto.ru/s/v1/ig2/AWuFFRYys_Q5i-kOo7V72eESQYMUXggJTNUaKyi0KIqP3D-tliyYYpQ9hGq6voVzWBa-_P_Kc2A4ofS1rr78ilf7.jpg?quality=95&as=32x43,48x64,72x96,108x144,160x213,240x320,360x480,480x640,540x720,640x853,720x960,1080x1440,1280x1707,1440x1920,1920x2560&from=bu',
 'width': 1920}

### `Пример с YouTube API`

```bash
pip install google-api-python-client
```

In [22]:
import os
import googleapiclient.discovery


os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = os.environ["youtube_api_token"]

youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=DEVELOPER_KEY)

request = youtube.commentThreads().list(
    part="id,snippet,replies",
    allThreadsRelatedToChannelId="UChLtMSiwB5Lip1adWm0k3nw",
    # maxResults=100,
    order="time",
    textFormat="plainText"
)
response = request.execute()

In [23]:
res = []
for com in response["items"]:
    comment_data = {}
    comment_data['comment_id'] = com['id']
    
    com_snippet = com['snippet']
    comment_data['channel_id'] = com_snippet['channelId']
    comment_data['video_id'] = com_snippet['videoId']
    
    com_snippet_toplevel_snippet = com['snippet']['topLevelComment']['snippet']
    comment_data['text'] = com_snippet_toplevel_snippet['textDisplay']
    comment_data['author_url'] = com_snippet_toplevel_snippet['authorChannelUrl']
    comment_data['author_id'] = com_snippet_toplevel_snippet['authorChannelId']['value']
    comment_data['like_count'] = com_snippet_toplevel_snippet['likeCount']
    comment_data['published_dt'] = com_snippet_toplevel_snippet['publishedAt']
    comment_data['updated_dt'] = com_snippet_toplevel_snippet['updatedAt']
    res.append(comment_data)

In [24]:
res[1], res[5]

({'comment_id': 'UgxGxCGmVXeWBe9Wqdt4AaABAg',
  'channel_id': 'UChLtMSiwB5Lip1adWm0k3nw',
  'video_id': 'aKRaqdF8apQ',
  'text': 'Дюна и с этим треком\n тоже прекрасно справилась))))',
  'author_url': 'http://www.youtube.com/@twenty_bro_five24',
  'author_id': 'UCDt30nw4obv9NnTBAagUI6w',
  'like_count': 0,
  'published_dt': '2024-12-12T05:35:55Z',
  'updated_dt': '2024-12-12T05:35:55Z'},
 {'comment_id': 'UgyG7rH8fxiyBfPMB3R4AaABAg',
  'channel_id': 'UChLtMSiwB5Lip1adWm0k3nw',
  'video_id': 'nQ2NBadFZGc',
  'text': 'Даже подавился яблоком от смеха😂😂😂😂',
  'author_url': 'http://www.youtube.com/@Pin2001-Channel',
  'author_id': 'UCSateqIyW9Su83TiVz64_fg',
  'like_count': 0,
  'published_dt': '2024-12-10T18:44:54Z',
  'updated_dt': '2024-12-10T18:44:54Z'})

### `Пример OpenAI API`

```bash
curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
     "model": "gpt-4o-mini",
     "messages": [{"role": "user", "content": "Say this is a test!"}],
     "temperature": 0.7
   }'
```

In [ ]:
import openai
from pprint import pprint

client = openai.OpenAI(
    base_url="http://localhost:8000/v1",
    api_key=os.environ["OPENAI_API_KEY"]
)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are ChatGPT, an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests."
        },
        {
            "role": "user",
            "content": "Write a limerick about python exceptions"
        }
    ]
)

pprint(completion.choices[0].message.content)

### `Состав запроса`

- HTTP-метод: GET, POST, OPTIONS и т. д., определяющее операцию, которую клиент хочет выполнить
- Путь к ресурсу
- Версию HTTP-протокола
- Заголовки  (опционально)
- Тело (для некоторых методов, таких как POST)


```
GET / HTTP/1.1
Host: ya.ru
User-Agent: Python script
Accept: */*

```

### `Состав ответа`

- Версию HTTP-протокола
- HTTP код состояния, сообщающий об успешности запроса или причине неудачи
- Сообщение состояния -- краткое описание кода состояния
- HTTP заголовки
- Опционально: тело, содержащее пересылаемый ресурс


```
HTTP/1.1 200 Ok
Cache-Control: no-cache,no-store,max-age=0,must-revalidate
Content-Length: 59978
Content-Type: text/html; charset=UTF-8
Date: Thu, 29 Apr 2021 03:48:39 GMT
Set-Cookie: yp=1622260119.ygu.1; Expires=Sun, 27-Apr-2031 03:48:39 GMT; Domain=.ya.ru; Path=/
```

### `Типы запросов`

Типы запросов в какой-то степени просто договорённость о семантике. Никто не мешает пользоваться только одним типом. Но глобально типы призваны определить что именно требуется при обращении к одному и тому же ресурсу. Каждый конкретный ресурс может поддерживать только часть методов.

- GET — запрашивает представление ресурса. Запросы с использованием этого метода могут только извлекать данные
- HEAD — запрашивает ресурс так же, как и метод GET, но без тела ответа
- POST — используется для отправки сущностей к определённому ресурсу. Часто вызывает изменение состояния или какие-то побочные эффекты на сервере
- PUT — заменяет все текущие представления ресурса данными запроса
- DELETE — удаляет указанный ресурс
- CONNECT — устанавливает "туннель" к серверу, определённому по ресурсу
- OPTIONS — используется для описания параметров соединения с ресурсом
- TRACE — выполняет вызов возвращаемого тестового сообщения с ресурса (например, для отладки)
- PATCH — используется для частичного изменения ресурса

### `Заголовки`

- Authentication (WWW/Proxy-Authenticate, Authorization, Proxy-Authorization)
- Caching (Cache-control, Age, Expires)
- Client hints
- Conditionals
- Connection management (Connection, Keep-Alive)
- Cookies
- Message body information (Content-Length, Content-Type, Content-Encoding)
- Request context (Host, Referer, User-Agent)
- Response context
- Security (Referrer-Policy)
- WebSockets


### `Запуск на localhost`

Попробуем себя в роли devops и развернем локально HTTP-сервер, который будет принимать текстовые запросы и выдавать ответы,сгенерированные LLM.


Склонируем репозиторий проекта llama.cpp:
```bash
git clone https://github.com/ggerganov/llama.cpp.git
cd llama.cpp/
```

Сбилдим проект:
```bash
make llama-server
```

Скачаем веса модели:
```bash
wget https://huggingface.co/bartowski/Llama-3.2-1B-Instruct-GGUF/resolve/main/Llama-3.2-1B-Instruct-Q4_K_M.gguf
```

Запустим сервер
```bash
~/repos/llama.cpp/llama-server \
    -m ~/repos/sirius-ai/day_3/web-dev/gguf/Llama-3.2-1B-Instruct-Q4_K_M.gguf \
    --port 8001 \
    --host 127.0.0.1 \
    --ctx-size 512
```

Теперь мы можем делать запросы к локальному серверу:
```bash
curl http://127.0.0.1:8001/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer sk-no-key-required" \
  -d '{
     "model": "-",
     "messages": [{"role": "user", "content": "tell me a joke"}],
     "temperature": 0.7
   }'
```


Более того, проект llama.cpp предоставляет и Web UI! Если открыть `http://127.0.0.1:8001` в браузере, то можно пообщаться с моделью в чат-интерфейсе.

### `Коды`

- Информационные (100 - 199)
- Успешные (200 - 299)
- Перенаправления (300 - 399)
- Клиентские ошибки (400 - 499)
- Серверные ошибки (500 - 599)

```
200 OK
302 Found
400 Bad Request
401 Unauthorized
404 Not Found
500 Internal Server Error
503 Service Unavailable
```

### `HTTPS`

- HTTPS не является отдельным протоколом передачи данных, а представляет собой расширение протокола HTTP с надстройкой шифрования
- передаваемые по протоколу HTTP данные не защищены, HTTPS обеспечивает конфиденциальность информации путем ее шифрования
- HTTP использует порт 80, HTTPS — порт 443

#### Сетевое обеспечение защищённого протокола

SSL — Secure Sockets Layer: "надстройка" на несколькими слоями сетевой модели

TLS — Transport Layer Security: развитие SSL

Обеспечивают шифрование и поддержку сертификатов

Принцип работы:
- С помощью ассиметричного шифрования устанавливается ключ соединения и передаётся сессионный ключ
- Всё дальнейшее общение шифруется уже сессионным ключом

#### Асимметричное шифрование

Называется так, потому что передающая сторона может только зашифровать (но не расшифровать) данные.

Асимметричный ключ — ключ, имеющий две составляющие: публичную и частную (закрытую). Публичный ключ доступен любому. Частный (закрытый) известен только владельцу. Если браузер хочет отправить сообщение, то он находит публичный ключ сервера, шифрует сообщение и отправляет на сервер. Далее сервер расшифровывает полученное сообщение с помощью своего частного ключа. Чтобы ответить пользователю, сервер делает те же самые действия: поиск публичного ключа собеседника, шифрование, отправка

Следовательно, для двустороннего общения требуется 2 пары ключей.

Алгоритмы ассиметричного шифрования более ресурсоёмкие, поэтому обычно только первичная установка соединения производится с его помощью, далее стороны договариваются о симметричном сессионном ключе и дальшейшее общение ведётся с помощью симметричного шифрованиия

#### Симметричное шифрование

В этом случае у обеих сторон есть один ключ, с помощью которого они и передают данные. Основная проблема — как договориться об этом ключе, чтобы знали только две стороны. И один из способов (не единственный) — с помощью ассиметричных алгоритмов